In [ ]:
!pip install requests beautifulsoup4 pandas numpy tqdm matplotlib seaborn python-dotenv torch scikit-learn webdriver_manager selenium

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import os
from datetime import datetime
import logging
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Load environment variables for GitHub API
load_dotenv()
GITHUB_TOKEN = open("github_api.key","r").read()
if not GITHUB_TOKEN:
    logger.error("GitHub token not found. Please set GITHUB_TOKEN in .env file")
    raise ValueError("GitHub token not found")



GITHUB_TOKEN=""




headers = {
    'Authorization': f'token {GITHUB_TOKEN}',
    'Accept': 'application/vnd.github.v3+json'
}

### List of Projects - from Apache website

In [10]:
project_list_data = open("Apache_projects.html","r").read()
# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(project_list_data, 'html.parser')

# Find all <a> tags
a_tags = soup.find_all('a')

# Extract the href and text from each <a> tag
links_and_names = [(a.text, a['href']) for a in a_tags if 'href' in a.attrs]

# Convert the list to a pandas DataFrame
df = pd.DataFrame(links_and_names, columns=['Name', 'Sublink'])

In [ ]:
def get_final_url(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the redirect to complete
    return driver.current_url


# Configure Chrome options
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Set up WebDriver using webdriver-manager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

for i, row in df[:].iterrows():
    url = "https://projects.apache.org/" + row["Sublink"]
    # Load the page and allow time for JS to execute
    driver.get(url)
    time.sleep(10)  # Increase if needed

    # Get the fully rendered page source
    page = driver.page_source

    # Parse with BeautifulSoup
    soup = BeautifulSoup(page, 'html.parser')
    
    # Find all <li> tags containing <b>Git repository:</b>
    li_tags = soup.find_all('li')
    git_repo_link = None
    for li in li_tags:
        b_tag = li.find('b')
        if b_tag and b_tag.text == 'Git repository:':
            a_tag = li.find('a', href=True)
            if a_tag:
                git_repo_link = a_tag['href']
                break
    
    project_status = None
    for li in li_tags:
        b_tag = li.find('b')
        if b_tag and b_tag.get_text(strip=True) == 'Project status:':
            # Prefer extracting text from the <span> tag
            span_tag = li.find('span')
            if span_tag:
                project_status = span_tag.get_text(strip=True)
            else:
                # Fallback extraction if span is missing
                project_status = li.get_text(strip=True).split(":", 1)[-1].strip()
            break

    # If the git_repo_link points to an .apache.org page, get its final URL
    if git_repo_link and '.apache.org' in git_repo_link:
        git_repo_link = get_final_url(driver, git_repo_link)
    
    print("Git Repository Link:", git_repo_link)
    print("Project Status:", project_status)
    
    # Update DataFrame columns (ensure these columns exist in df)
    df.at[i, "Github_link"] = git_repo_link
    df.at[i, "Project_status"] = project_status

driver.quit()

# Print and save the DataFrame to CSV
print(df)
df.to_csv("Apache_projects.csv", index=False)

2025-03-12 07:35:05,237 - INFO - ====== WebDriver manager ======
2025-03-12 07:35:06,138 - INFO - Get LATEST chromedriver version for google-chrome
2025-03-12 07:35:06,609 - INFO - Get LATEST chromedriver version for google-chrome
2025-03-12 07:35:10,395 - INFO - Driver [C:\Users\abhijeet\.wdm\drivers\chromedriver\win64\134.0.6998.88\chromedriver-win32/chromedriver.exe] found in cache


Git Repository Link: https://github.com/apache/db-jdo
Project Status: Active
Git Repository Link: https://github.com/apache/ant-antlibs-dotnet
Project Status: Active
Git Repository Link: https://github.com/apache/accumulo
Project Status: Active
Git Repository Link: None
Project Status: Active
Git Repository Link: https://github.com/apache/age
Project Status: Active
Git Repository Link: https://github.com/apache/airavata
Project Status: Active
Git Repository Link: https://github.com/apache/airflow.git
Project Status: Active
Git Repository Link: None
Project Status: Active
Git Repository Link: https://github.com/apache/ambari
Project Status: Active
Git Repository Link: None
Project Status: Incubating
Git Repository Link: None
Project Status: Active
Git Repository Link: http://git@github.com:apache/incubator-annotator.git
Project Status: Incubating
Git Repository Link: None
Project Status: Active
Git Repository Link: https://github.com/apache/ant
Project Status: Active
Git Repository Link

In [13]:
df = pd.read_csv("Apache_projects.csv")
df

,Name,Sublink,Github_link,Project_status
0,Apache JDO,project.html?db-_jdo,https://github.com/apache/db-jdo,Active
1,Apache .NET Ant Library,project.html?ant-dotnet,https://github.com/apache/ant-antlibs-dotnet,Active
2,Apache Accumulo,project.html?accumulo,https://github.com/apache/accumulo,Active
3,Apache ActiveMQ,project.html?activemq,NaN,Active
4,Apache AGE,project.html?age,https://github.com/apache/age,Active
...,...,...,...,...
382,Apache XTable (Incubating),project.html?incubator-xtable,NaN,NaN
383,Apache Yetus,project.html?yetus,NaN,NaN
384,Apache YuniKorn,project.html?yunikorn,NaN,NaN
385,Apache Zeppelin,project.html?zeppelin,NaN,NaN


### List of Projects - from Apache incubator projects

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

def parse_project_table(soup, section_id, project_type):
    """
    Parse a project table from a given section header (section_id) and assign a project type.
    Returns a DataFrame with columns:
      - id: the <tr> element's id attribute
      - href: the href from the first <a> in the first <td> that starts with "/projects/"
      - type: the project type provided (e.g., "Graduated Projects", "Retired Podlings")
      - Apache Sponsor: text from the corresponding column
      - Mentors: text from the corresponding column
      - Start Date: text from the corresponding column
      - End Date: text from the corresponding column
    """
    header = soup.find("h3", id=section_id)
    if header is None:
        print(f"Section with id '{section_id}' not found.")
        return pd.DataFrame()
    
    table = header.find_next("table")
    if table is None:
        print(f"No table found after section '{section_id}'.")
        return pd.DataFrame()
    
    rows = table.find_all("tr")
    data = []
    # Skip header row (first row)
    for row in rows[1:]:
        row_id = row.get("id", "").strip()
        cells = row.find_all("td")
        # Ensure that there are at least 7 cells
        if len(cells) < 7:
            continue
        
        first_cell = cells[0]
        a_tag = first_cell.find("a", href=True)
        href = a_tag["href"] if a_tag and a_tag["href"].startswith("/projects/") else ""
        
        apache_sponsor = cells[3].get_text(strip=True)
        mentors = cells[4].get_text(strip=True)
        start_date = cells[5].get_text(strip=True)
        end_date = cells[6].get_text(strip=True)
        
        data.append({
            "id": row_id,
            "href": href,
            "type": project_type,
            "Apache Sponsor": apache_sponsor,
            "Mentors": mentors,
            "Start Date": start_date,
            "End Date": end_date
        })
    
    return pd.DataFrame(data)

def extract_all_repo_links(project_url):
    """
    Given the full URL of a project page, fetch the page and extract all:
      - GitHub links (where the href contains "github.com")
      - Gitbox links (where the href contains "gitbox")
    Returns two lists: (list_of_github_links, list_of_gitbox_links).
    If none are found, the lists will be empty.
    """
    try:
        response = requests.get(project_url)
        response.raise_for_status()
        page_soup = BeautifulSoup(response.text, "html.parser")
    except Exception as e:
        print(f"Error fetching {project_url}: {e}")
        return [], []
    
    github_links = []
    gitbox_links = []
    
    for a in page_soup.find_all("a", href=True):
        href = a["href"]
        if "github.com" in href:
            github_links.append(href)
        if "gitbox" in href:
            gitbox_links.append(href)
    
    return github_links, gitbox_links

def main():
    # Read the HTML content from the file
    with open("Apache_incubator_projects.html", "r", encoding="utf-8") as f:
        html_content = f.read()
        
    soup = BeautifulSoup(html_content, "html.parser")
    
    # Parse the two project tables
    df_graduated = parse_project_table(soup, "graduated", "Graduated Projects")
    df_retired   = parse_project_table(soup, "retired", "Retired Podlings")
    
    # Combine DataFrames
    combined_df = pd.concat([df_graduated, df_retired], ignore_index=True)
    
    # We'll store the lists of repo links in two new columns
    combined_df["github_links"] = ''
    combined_df["gitbox_links"] = ''
    
    base_url = "https://incubator.apache.org"
    
    for idx, row in combined_df.iterrows():
        href = row["href"]
        if not href:
            combined_df.at[idx, "github_links"] = []
            combined_df.at[idx, "gitbox_links"] = []
            continue
        project_url = base_url + href
        print(f"Processing: {project_url}")
        github_list, gitbox_list = extract_all_repo_links(project_url)
        combined_df.at[idx, "github_links"] = github_list
        combined_df.at[idx, "gitbox_links"] = gitbox_list
        # Be polite to the server
        time.sleep(1)
    
    # Determine the maximum number of GitHub and Gitbox links across all projects
    max_github = max(len(links) for links in combined_df["github_links"])
    max_gitbox = max(len(links) for links in combined_df["gitbox_links"])
    
    # For each index, create separate columns for each GitHub link (github_link1, github_link2, ...)
    for i in range(max_github):
        col_name = f"github_link{i+1}"
        combined_df[col_name] = combined_df["github_links"].apply(lambda links: links[i] if len(links) > i else "")
    
    # Similarly, create separate columns for each Gitbox link (gitbox_link1, gitbox_link2, ...)
    for i in range(max_gitbox):
        col_name = f"gitbox_link{i+1}"
        combined_df[col_name] = combined_df["gitbox_links"].apply(lambda links: links[i] if len(links) > i else "")
    
    # Optionally, drop the temporary list columns
    combined_df.drop(columns=["github_links", "gitbox_links"], inplace=True)
    
    # For demonstration, print the updated DataFrame
    print("\nUpdated DataFrame with individual GitHub and Gitbox link columns:")
    print(combined_df.to_string(index=False))
    
    # Save the DataFrame to a CSV file
    combined_df.to_csv("Apache_incubator_projects.csv", index=False)
    
    return combined_df

main()


Processing: https://incubator.apache.org/projects/abdera.html
Processing: https://incubator.apache.org/projects/accumulo.html
Processing: https://incubator.apache.org/projects/ace.html
Processing: https://incubator.apache.org/projects/activemq.html
Processing: https://incubator.apache.org/projects/age.html
Processing: https://incubator.apache.org/projects/airavata.html
Processing: https://incubator.apache.org/projects/airflow.html
Processing: https://incubator.apache.org/projects/allura.html
Processing: https://incubator.apache.org/projects/ambari.html
Processing: https://incubator.apache.org/projects/amber.html
Processing: https://incubator.apache.org/projects/answer.html
Processing: https://incubator.apache.org/projects/any23.html
Processing: https://incubator.apache.org/projects/apex.html
Processing: https://incubator.apache.org/projects/apisix.html
Processing: https://incubator.apache.org/projects/apollo.html
Processing: https://incubator.apache.org/projects/aries.html
Processing: 

,id,href,type,Apache Sponsor,Mentors,Start Date,End Date,github_link1,github_link2,github_link3,...,gitbox_link1,gitbox_link2,gitbox_link3,gitbox_link4,gitbox_link5,gitbox_link6,gitbox_link7,gitbox_link8,gitbox_link9,gitbox_link10
0,abdera,/projects/abdera.html,Graduated Projects,Incubator,"Garrett Rooney, Paul Querna",2006-06-06,2008-11-08,,,,...,,,,,,,,,,
1,accumulo,/projects/accumulo.html,Graduated Projects,Incubator,"Benson Margulies, Alan Cabrera, Bernd Fondermann",2011-09-12,2012-03-21,,,,...,,,,,,,,,,
2,ace,/projects/ace.html,Graduated Projects,Board,"Niclas Hedhman, Carsten Ziegeler, Bertrand Del...",2009-04-24,2011-12-21,,,,...,,,,,,,,,,
3,activemq,/projects/activemq.html,Graduated Projects,Geronimo,James Strachan,2005-12-01,2007-02-06,,,,...,,,,,,,,,,
4,age,/projects/age.html,Graduated Projects,Incubator(Jim Jagielski),"Kevin Ratnasekera, Von Gosling, Felix Cheung, ...",2020-04-29,2022-05-18,,,,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,xap,/projects/xap.html,Retired Podlings,Incubator,"Cliff Schmidt, Robert Burrell Donkin, Susan Wu",2006-05-22,2009-06-26,,,,...,,,,,,,,,,
325,xmlbeanscxx,/projects/xmlbeanscxx.html,Retired Podlings,Web Services,Cliff Schmidt,2005-06-06,2007-05-16,,,,...,,,,,,,,,,
326,yoko,/projects/yoko.html,Retired Podlings,Geronimo,"Ken Coar, Geir Magnusson Jr., James Strachan",2006-01-31,2007-12-17,,,,...,,,,,,,,,,
327,zetacomponents,/projects/zetacomponents.html,Retired Podlings,Incubator,"Julien Vermillard, Christian Grobmeier, Craig ...",2010-05-21,2012-04-20,http://github.com/zetacomponents,,,...,,,,,,,,,,


### List of Projects - from Apache github projects

In [ ]:
import requests
import pandas as pd
import time
from tqdm import tqdm

# GitHub API URL and token
GITHUB_API_URL = "https://api.github.com"

# Headers for API Authentication
HEADERS = {'Authorization': f'token {GITHUB_TOKEN}'}

def fetch_repositories(org):
    """
    Fetch all repositories for a given organization
    """
    repos = []
    page = 1
    while True:
        url = f"{GITHUB_API_URL}/orgs/{org}/repos?page={page}&per_page=100"
        response = requests.get(url, headers=HEADERS)
        if response.status_code != 200:
            print(f"Failed to fetch repositories: {response.status_code}")
            break
        data = response.json()
        if not data:
            break
        repos.extend(data)
        page += 1
        time.sleep(1)  # To avoid hitting rate limits
    return repos

def extract_repo_info(repos):
    """
    Extract id, name, and url from repository data
    """
    repo_info = []
    for repo in repos:
        repo_info.append({
            'id': repo['id'],
            'node_id': repo['node_id'],
            'owner': repo['owner']['login'],
            'name': repo['name'],
            'url': repo['html_url'],
            'created_at': repo['created_at'],
            'updated_at': repo['updated_at'],
            'pushed_at': repo['pushed_at'],
            'size': repo['size'],
            'stargazers_count': repo['stargazers_count'],
            #'watchers_count': repo['watchers_count'],
            'language': repo['language'],
            'has_issues': repo['has_issues'],
            'has_projects': repo['has_projects'],
            'has_downloads': repo['has_downloads'],
            'has_wiki': repo['has_wiki'],
            'has_pages': repo['has_pages'],
            'has_discussions': repo['has_discussions'],
            'forks_count': repo['forks_count'],
            'mirror_url': repo['mirror_url'],
            'archived': repo['archived'],
            'disabled': repo['disabled'],
            'open_issues_count': repo['open_issues_count'],
            'license': repo['license'],
            'allow_forking': repo['allow_forking'],
            'is_template': repo['is_template'],
            'web_commit_signoff_required': repo['web_commit_signoff_required'],
            'forks': repo['forks'],
            'open_issues': repo['open_issues'],
            'watchers': repo['watchers'],
            'default_branch': repo['default_branch'],
            'permissions_admin': repo['permissions']['admin'],
            'permissions_maintain': repo['permissions']['maintain'],
            'permissions_push': repo['permissions']['push'],
            'permissions_pull': repo['permissions']['pull'],
        })
    return repo_info

def save_to_csv(data, filename):
    """
    Save data to a CSV file
    """
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

# Fetch repositories for a given organization
organization = "apache"  # Replace with the desired organization
repositories = fetch_repositories(organization)

# Extract repository information
repo_info = extract_repo_info(repositories)

# Save the repository information to a CSV file
save_to_csv(repo_info, "Apache_github_repositories.csv")

Data saved to Apache_github_repositories.csv


### Combining all 3 lists

In [45]:
import pandas as pd

# Load your CSV files
incubator_data = pd.read_csv("Apache_incubator_projects.csv")
apache_project_data = pd.read_csv("Apache_projects.csv")
github_project_data = pd.read_csv("Apache_github_repositories.csv")

# Create standardized name columns for matching.
# For incubator_data, assume the name is in the column 'id'.
incubator_data['name_std'] = incubator_data['id'].astype(str).str.strip().str.lower()
# For apache_project_data, assume we need to drop the first 7 characters from the 'Name' column.
apache_project_data['name_std'] = apache_project_data['Name'].astype(str).str[7:].str.strip().str.lower()
# For github_project_data, assume the name is in the column 'name'
github_project_data['name_std'] = github_project_data['name'].astype(str).str.strip().str.lower()

# Assume the following for GitHub links:
# In apache_project_data, the column with the link is "Github_link"
# In github_project_data, the GitHub URL is in the column "url" (adjust if necessary)

# Define the incubator columns to check for GitHub links.
incubator_link_cols = [
    "github_link1", "github_link2", "github_link3", "github_link4",
    "github_link5", "github_link6", "github_link7", "github_link8",
    "github_link9", "github_link10", "github_link11", "github_link12",
    "github_link13", "gitbox_link1", "gitbox_link2", "gitbox_link3",
    "gitbox_link4", "gitbox_link5", "gitbox_link6", "gitbox_link7",
    "gitbox_link8", "gitbox_link9", "gitbox_link10"
]

# Prepare column names with prefixes so that they are distinguishable in the final DataFrame.
def prefixed_cols(df, prefix):
    return [f"{prefix}_{col}" for col in df.columns if col != "name_std"]

github_cols = prefixed_cols(github_project_data, "github")
apache_cols = prefixed_cols(apache_project_data, "apache")
incubator_cols = prefixed_cols(incubator_data, "incubator")

# The combined DataFrame will have columns from all three sets.
combined_columns = github_cols + apache_cols + incubator_cols

# Instead of using DataFrame.append (removed in newer pandas), collect rows as dictionaries.
rows = []

# Loop over GitHub rows and try to match rows in Apache and Incubator dataframes.
for idx, github_row in github_project_data.iterrows():
    name = github_row['name_std']
    combined_row = {}
    
    # Add GitHub data (all columns except name_std) with prefix "github_".
    for col in github_project_data.columns:
        if col != 'name_std':
            combined_row[f'github_{col}'] = github_row[col]
    
    # --- Apache matching ---
    # First try to match by standardized name.
    apache_match = apache_project_data[apache_project_data['name_std'] == name]
    # If no name match, check if Apache's Github_link equals github_row['url'].
    if apache_match.empty:
        apache_match = apache_project_data[apache_project_data['Github_link'] == github_row['url']]
    
    if not apache_match.empty:
        match_idx = apache_match.index[0]
        apache_row = apache_project_data.loc[match_idx]
        for col in apache_project_data.columns:
            if col != 'name_std':
                combined_row[f'apache_{col}'] = apache_row[col]
        # Remove the matched row from apache_project_data.
        apache_project_data = apache_project_data.drop(match_idx)
    else:
        # No match found; fill with NA for each Apache column.
        for col in apache_project_data.columns:
            if col != 'name_std':
                combined_row[f'apache_{col}'] = pd.NA

    # --- Incubator matching ---
    # First try to match by standardized name.
    incubator_match = incubator_data[incubator_data['name_std'] == name]
    # If no name match, check if the github URL is in any of the incubator link columns.
    if incubator_match.empty:
        incubator_match = incubator_data[incubator_data.apply(
            lambda row: any(github_row['url'] == str(row[col]).strip() for col in incubator_link_cols if pd.notna(row[col])),
            axis=1)]
    
    if not incubator_match.empty:
        match_idx = incubator_match.index[0]
        incubator_row = incubator_data.loc[match_idx]
        for col in incubator_data.columns:
            if col != 'name_std':
                combined_row[f'incubator_{col}'] = incubator_row[col]
        # Remove the matched row from incubator_data.
        incubator_data = incubator_data.drop(match_idx)
    else:
        # No match; fill with NA for each incubator column.
        for col in incubator_data.columns:
            if col != 'name_std':
                combined_row[f'incubator_{col}'] = pd.NA

    # Append the combined row dictionary to our list.
    rows.append(combined_row)

# Append any remaining Apache rows that were not matched.
for idx, apache_row in apache_project_data.iterrows():
    combined_row = {}
    # Fill GitHub columns with NA.
    for col in github_project_data.columns:
        if col != 'name_std':
            combined_row[f'github_{col}'] = pd.NA
    # Add Apache data.
    for col in apache_project_data.columns:
        if col != 'name_std':
            combined_row[f'apache_{col}'] = apache_row[col]
    # Fill Incubator columns with NA.
    for col in incubator_data.columns:
        if col != 'name_std':
            combined_row[f'incubator_{col}'] = pd.NA
    rows.append(combined_row)

# Append any remaining Incubator rows that were not matched.
for idx, incubator_row in incubator_data.iterrows():
    combined_row = {}
    # Fill GitHub and Apache columns with NA.
    for col in github_project_data.columns:
        if col != 'name_std':
            combined_row[f'github_{col}'] = pd.NA
    for col in apache_project_data.columns:
        if col != 'name_std':
            combined_row[f'apache_{col}'] = pd.NA
    # Add Incubator data.
    for col in incubator_data.columns:
        if col != 'name_std':
            combined_row[f'incubator_{col}'] = incubator_row[col]
    rows.append(combined_row)

# Create the final DataFrame from the accumulated rows with the specified column order.
result_df = pd.DataFrame(rows, columns=combined_columns)

# --- Add final_status column ---
def get_final_status(row):
    # 1. Check incubator type, assumed to be in 'incubator_incubator_type'
    if pd.notna(row.get('incubator_type', None)):
        if row['incubator_type'] == "Graduated Projects":
            return 1
        elif row['incubator_type'] == "Retired Podlings":
            return 0
    # 2. Else, check apache_Project_status, assumed to be in 'apache_Project_status'
    if pd.notna(row.get('apache_Project_status', None)):
        status = row['apache_Project_status']
        if status == "Incubating":
            return -1
        elif status == "Active":
            return 1
        elif status == "Retired":
            return 0
    # 3. Else, check github_archived, assumed to be in 'github_github_archived'
    if pd.notna(row.get('github_archived', None)):
        archived = row['github_archived']
        # Assuming archived is boolean or a string representation.
        if archived in [True, 'True', 'true', 1]:
            return 0
        else:
            return 1
    return pd.NA

# Apply the function to each row to create the final_status column.
result_df['final_status'] = result_df.apply(get_final_status, axis=1)

# Move final_status to be the leftmost column.
cols = result_df.columns.tolist()
cols.insert(0, cols.pop(cols.index('final_status')))
result_df = result_df[cols]

# Save the resulting DataFrame to a CSV file (optional).
result_df.to_csv("R_Apache_combined_data.csv", index=False)



     final_status github_id                github_node_id github_owner  \
0               0    160986  MDEwOlJlcG9zaXRvcnkxNjA5ODY=       apache   
1               1    160988  MDEwOlJlcG9zaXRvcnkxNjA5ODg=       apache   
2               0    160989  MDEwOlJlcG9zaXRvcnkxNjA5ODk=       apache   
3               1    160994  MDEwOlJlcG9zaXRvcnkxNjA5OTQ=       apache   
4               1    160995  MDEwOlJlcG9zaXRvcnkxNjA5OTU=       apache   
...           ...       ...                           ...          ...   
3020            0      <NA>                          <NA>         <NA>   
3021            0      <NA>                          <NA>         <NA>   
3022            0      <NA>                          <NA>         <NA>   
3023            0      <NA>                          <NA>         <NA>   
3024            0      <NA>                          <NA>         <NA>   

               github_name                                      github_url  \
0                tapestry3       

In [47]:
import pandas as pd

# Read the combined CSV file
df = pd.read_csv("R_Apache_combined_data.csv")

# Filter rows that have a non-null "github_name" and at least one of "apache_Name" or "incubator_id"
df_filtered = df[
    df['github_name'].notna() &
    (df['apache_Name'].notna() | df['incubator_id'].notna())
]

# Sort the filtered rows based on "github_size"
df_sorted = df_filtered.sort_values(by="github_size")

# Split the sorted data into two groups based on final_status (0 and 1)
group_0 = df_sorted[df_sorted['final_status'] == 0]
group_1 = df_sorted[df_sorted['final_status'] == 1]

# Determine the smaller group count to balance the dataset
min_count = min(len(group_0), len(group_1))

# Select the top min_count rows from each group (as the data is sorted by github_size)
balanced_0 = group_0.tail(min_count)
balanced_1 = group_1.tail(min_count)

# Concatenate the balanced groups
balanced_df = pd.concat([balanced_0, balanced_1]).sort_values(by="github_size")

# (Optional) Save the balanced dataframe to a CSV file
balanced_df.to_csv("R_Apache_balanced_data.csv", index=False)



In [ ]:
import requests
import pandas as pd
import time
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

# GitHub API URL and token
GITHUB_API_URL = "https://api.github.com"  # Replace with your GitHub token

# Headers for API Authentication
HEADERS = {'Authorization': f'token {GITHUB_TOKEN}'}

def get_with_retries(url, headers, retries=3, timeout=10):
    for i in range(retries):
        try:
            response = requests.get(url, headers=headers, timeout=timeout)
            response.raise_for_status()
            return response
        except requests.exceptions.RequestException as error:
            print(f"Attempt {i+1} failed: {error}")
            time.sleep(2**i)  # Exponential backoff
    print(f"Failed to fetch {url} after {retries} attempts")
    #raise Exception(f"Failed to fetch {url} after {retries} attempts")

def fetch_metric_count(owner, repo, start_date, end_date, metric_type):
    """
    Fetch the count of commits, issues, or pull requests based on the metric type and date filter.
    """
    url = ""
    if metric_type == "commits":
        url = f"{GITHUB_API_URL}/repos/{owner}/{repo}/commits?since={start_date}&until={end_date}"
    elif metric_type == "issues":
        query = f"repo:{owner}/{repo}+type:issue+created:{start_date}..{end_date}"
        url = f"{GITHUB_API_URL}/search/issues?q={query}"
    elif metric_type == "pulls":
        query = f"repo:{owner}/{repo}+type:pr+created:{start_date}..{end_date}"
        url = f"{GITHUB_API_URL}/search/issues?q={query}"

    try:
        print(url)
        response = get_with_retries(url, HEADERS, retries=2, timeout=10)
        data = response.json()  # Get JSON from the response
        print(data)
        if metric_type == "commits":
            return len(data)  # Commits API returns a list
        if metric_type == "issues" or metric_type == "pulls":
            return data['total_count']
        else:
            return data.get('total_count', 0)  # Issues and PRs return an object with total_count
    except requests.exceptions.RequestException as error:
        print(f"⚠️ Error fetching {metric_type} for {start_date}: {error}")
        return 0

def fetch_monthly_metrics(owner, repo, start_date, end_date, years=2):
    """
    Fetch the number of commits, issues, and pull requests per month 
    from the given start_date until the end_date or for the specified number of years at max.
    The month is formatted in "YYMM" format.
    """
    monthly_data = []
    # Current month as an offset-aware datetime
    now = end_date.replace(day=1)
    # Compute the earliest date based on the specified number of years
    max_years_ago = now - relativedelta(years=years)
    # Start at the later of the repository creation date or max_years_ago
    current = max(start_date.replace(day=1), max_years_ago)

    while current < now:
        next_month = current + relativedelta(months=1)
        # For commits, use 'since' and 'until' parameters.
        commits = fetch_metric_count(owner, repo, current.date().isoformat(), next_month.date().isoformat(), "commits")
        
        # For issues and PRs, use the 'created:' filter.
        issues = fetch_metric_count(owner, repo, current.date().isoformat(), next_month.date().isoformat(), "issues")
        pulls = fetch_metric_count(owner, repo, current.date().isoformat(), next_month.date().isoformat(), "pulls")
        
        # Format month as YYMM.
        month_str = current.strftime("%y%m")
        monthly_data.append({
            "month": month_str,
            "commits": commits,
            "issues": issues,
            "pulls": pulls
        })
        
        current = next_month
        time.sleep(1)  # Sleep to help avoid rate limiting

    return monthly_data
def main():
    # Load the DataFrame containing the repository URLs (adjust file name if needed)
    df = pd.read_csv("R_Apache_balanced_data.csv")
    
    # Open the JSON file in append mode or create it if it doesn't exist
    try:
        with open("repository_metrics.json", "r") as f:
            repo_metrics_list = json.load(f)
            processed_repos = {entry["Github_link"] for entry in repo_metrics_list}
    except FileNotFoundError:
        repo_metrics_list = []
        processed_repos = set()
    
    for i, row in tqdm(df[:].iterrows(), total=df.shape[0]):
        if pd.isna(row["github_url"]):
            continue
        
        # Skip repositories that have already been processed
        if row["github_url"] in processed_repos:
            continue
        
        # Extract owner and repo from the URL.
        try:
            owner, repo = row["github_url"].split("/")[-2], row["github_url"].split("/")[-1]
        except IndexError:
            print(f"Invalid URL format: {row['github_url']}")
            continue

        creation_date = datetime.fromisoformat(row["github_created_at"].replace("Z", "+00:00"))
        end_date = datetime.fromisoformat(row["github_pushed_at"].replace("Z", "+00:00"))
        print(f"{creation_date} ---- {end_date}")
        monthly_metrics = fetch_monthly_metrics(owner, repo, creation_date, end_date, years=2)
        
        repo_entry = {
            "name": row["github_name"],
            "Github_link": row["github_url"],
            "Graduated": row["final_status"],
            "start_date": creation_date.strftime("%Y-%m-%d"),
            "monthly_metrics": monthly_metrics
        }
        repo_metrics_list.append(repo_entry)
        
        # Save the updated list of repository metrics to the JSON file after each iteration
        with open("repository_metrics.json", "w") as f:
            json.dump(repo_metrics_list, f, indent=4)
    
    print("Data saved to repository_metrics.json")

if __name__ == "__main__":
    main()


### main model

#### creates testing files

In [ ]:
import json
import random
import pandas as pd

# Load the JSON file
with open("repository_metrics.json", "r") as f:
    data = json.load(f)


# Add a new parameter "labels" to each entry in "monthly_metrics"
for project in data:
    for metric in project.get("monthly_metrics", []):
        metric["labels"] = random.randint(1, 7)  # Randomly assign a value between 1 and 7

df = pd.read_csv("R_Apache_balanced_data.csv")
status_mapping = dict(zip(df["github_url"], df["final_status"]))

# For each project, add final_status from CSV using Github_link as the key
for project in data:
    github_link = project.get("Github_link")
    if github_link in status_mapping:
        project["final_status"] = status_mapping[github_link]

# Save the updated JSON
with open("repository_metrics_label.json", "w") as f:
    json.dump(data, f, indent=4)

print("Updated JSON saved to repository_metrics_label.json")

In [34]:
import tensorflow as tf
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Masking
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam

# 1. Load and prepare the data
def load_data(json_path):
    """
    Load JSON data with monthly metrics and CSV with final status
    """
    # Load projects data with monthly categorical metrics
    with open(json_path, 'r') as f:
        projects_data = json.load(f)
    
    return projects_data

def one_hot(label, num_classes=7):
    vec = [0] * num_classes
    # Convert label (1-indexed) to 0-indexed vector position
    vec[label - 1] = 1
    return vec


# Main execution
def main():
    # Paths to data files
    label_json = 'repository_metrics_label.json'  # Path to your JSON file with projects data
        
    # Load the CSV files as part of the pipeline
    projects_data = load_data(label_json)
    num_classes = 7

    window_size = 3 
    X_data = []
    y_data = [] 
        
    for project in projects_data:
        y_label = project.get("final_status", [])
        monthly_metrics = project.get("monthly_metrics", [])
        # Optional: sort monthly_metrics by month if not already sorted
        monthly_metrics = sorted(monthly_metrics, key=lambda m: m["month"])
        
        # Extract labels and convert them to one-hot encoded vectors
        labels = [m["labels"] for m in monthly_metrics]
        one_hot_labels = [one_hot(label) for label in labels]
        
        # Create sliding window sequences
        # For each possible window, the next time step becomes the ground truth
        for i in range(len(one_hot_labels) - window_size):
            X_seq = one_hot_labels[i : i + window_size]  # Input sequence
            X_data.append(X_seq)
            y_data.append(y_label)

    X_data = np.array(X_data)
    y_data = np.array(y_data)

    print("Input shape (X):", X_data.shape)
    print("Ground truth shape (y):", y_data.shape)
     
    # Build the LSTM model
    model = Sequential()
    # The LSTM input shape is (window_size, num_features)
    model.add(LSTM(128, input_shape=(window_size, num_classes)))
    model.add(Dropout(0.2))
    # Output layer: number of classes with softmax activation for classification
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    
    model.summary()
    
    # Train the model
    epochs = 250
    batch_size = 32
    model.fit(X_data, y_data, epochs=epochs, batch_size=batch_size, validation_split=0.3)


if __name__ == "__main__":
    main()

Input shape (X): (546, 3, 7)
Ground truth shape (y): (546,)


c:\Users\abhijeet\miniconda3\envs\tf\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_10 (LSTM)                  │ (None, 128)            │        69,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 69,761 (272.50 KB)

 Trainable params: 69,761 (272.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/250
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - accuracy: 0.6689 - loss: 0.6765 - val_accuracy: 0.0000e+00 - val_loss: 0.8429
Epoch 2/250
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7272 - loss: 0.6290 - val_accuracy: 0.0000e+00 - val_loss: 1.0723
Epoch 3/250
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7320 - loss: 0.5845 - val_accuracy: 0.0000e+00 - val_loss: 1.3624
Epoch 4/250
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7076 - loss: 0.5875 - val_accuracy: 0.0000e+00 - val_loss: 1.3465
Epoch 5/250
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7075 - loss: 0.5803 - val_accuracy: 0.0000e+00 - val_loss: 1.3553
Epoch 6/250
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7392 - loss: 0.5541 - val_accuracy: 0.0000e+00 - val_loss: 1.3562
Epoch 7/250
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7164 - loss: 0.5707 - val_accuracy: 0.0000e+00 - val_loss: 1.4114
Epoch 8/250
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7112 - loss: 0.